言語モデルはテキストを出力します。しかし、多くの場合、ただのテキストよりももっと構造化された情報が欲しいと思うことがあります。そこで出力パーサーが役立ちます。

> Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

出力パーサーは、言語モデルの応答を構造化するのに役立つクラスです。出力パーサーが実装する必要がある二つの主要なメソッドがあります：

> Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

* "Get format instructions": 言語モデルの出力がどのようにフォーマットされるべきかについての指示を含む文字列を返すメソッド。

  > "Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.

* 「Parse」: 文字列を入力として受け取り（言語モデルからの応答であると想定される）、それをある構造に解析するメソッド。

  > "Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.


そして、もう一つの任意の選択肢があります：

> And then one optional one:

* 'Parse with prompt'は、文字列（言語モデルからの応答と想定される）とプロンプト（そのような応答を生成したと想定されるプロンプト）を受け取り、それを何らかの構造に解析するメソッドです。プロンプトは、OutputParserが出力を再試行するか、何らかの方法で修正する必要がある場合、そのための情報をプロンプトから得るために主に提供されます。

  > "Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.


## Get started | 始めましょう

以下では、主要な出力パーサータイプである `PydanticOutputParser` について説明します。

> Below we go over the main type of output parser, the `PydanticOutputParser`.




In [6]:
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

## LCEL | LCEL

出力パーサーは、[LangChain Expression Language (LCEL)](/docs/expression_language/)の基本的な構成要素である[Runnable interface](/docs/expression_language/interface)を実装しています。これにより、`invoke`、`ainvoke`、`stream`、`astream`、`batch`、`abatch`、`astream_log`の各メソッドをサポートしていることを意味します。

> Output parsers implement the [Runnable interface](/docs/expression_language/interface), the basic building block of the [LangChain Expression Language (LCEL)](/docs/expression_language/). This means they support `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` calls.

出力パーサーは文字列または`BaseMessage`を入力として受け取り、任意の型を返すことができます。

> Output parsers accept a string or `BaseMessage` as input and can return an arbitrary type.




In [7]:
parser.invoke(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

パーサーを手動で起動する代わりに、私たちはそれを`Runnable`シーケンスに追加することもできましたが：

> Instead of manually invoking the parser, we also could've just added it to our `Runnable` sequence:




In [8]:
chain = prompt | model | parser
chain.invoke({"query": "Tell me a joke."})

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

すべてのパーサーはストリーミングインターフェースをサポートしていますが、部分的に解析されたオブジェクトを通じてストリーミングできるパーサーは限られています。これは出力タイプに大きく依存するためです。部分的なオブジェクトを構築できないパーサーは、完全に解析された出力を単に提供します。

> While all parsers support the streaming interface, only certain parsers can stream through partially parsed objects, since this is highly dependent on the output type. Parsers which cannot construct partial objects will simply yield the fully parsed output.

`SimpleJsonOutputParser` は、例えば部分的な出力を逐次的に処理できます：

> The `SimpleJsonOutputParser` for example can stream through partial outputs:




In [16]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

In [17]:
list(json_chain.stream({"question": "Who invented the microscope?"}))

[{},
 {'answer': ''},
 {'answer': 'Ant'},
 {'answer': 'Anton'},
 {'answer': 'Antonie'},
 {'answer': 'Antonie van'},
 {'answer': 'Antonie van Lee'},
 {'answer': 'Antonie van Leeu'},
 {'answer': 'Antonie van Leeuwen'},
 {'answer': 'Antonie van Leeuwenho'},
 {'answer': 'Antonie van Leeuwenhoek'}]

PydanticOutputParserは以下のことができません：

> While the PydanticOutputParser cannot:




In [18]:
list(chain.stream({"query": "Tell me a joke."}))

[Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')]